# Book Catalog API Testing

This notebook comprehensively tests the Book Catalog API built with FastAPI.

## API Endpoints Overview
- `GET /` - API root and information
- `GET /health` - Health check
- `POST /books/` - Create a new book
- `GET /books/` - List books with filtering/pagination
- `GET /books/{id}` - Get specific book by ID
- `DELETE /books/{id}` - Delete book by ID
- `GET /books/stats/summary` - Get collection statistics

## Test Categories
1. Basic connectivity and health checks
2. Book creation with validation testing
3. Book retrieval and filtering
4. Pagination and sorting
5. Individual book operations
6. Statistics endpoint
7. Error handling scenarios

In [ ]:
pip install requests


In [ ]:
pip install pandas

In [ ]:
import requests
import json
from datetime import datetime
from typing import Dict, List, Optional
import pandas as pd
from pprint import pprint

# Configuration
BASE_URL = "http://localhost:8000"
HEADERS = {"Content-Type": "application/json"}

# Helper function for making requests
def make_request(method: str, endpoint: str, data: Optional[Dict] = None, params: Optional[Dict] = None) -> requests.Response:
    """Helper function to make API requests with consistent error handling"""
    url = f"{BASE_URL}{endpoint}"
    try:
        if method.upper() == "GET":
            response = requests.get(url, params=params, headers=HEADERS)
        elif method.upper() == "POST":
            response = requests.post(url, json=data, headers=HEADERS)
        elif method.upper() == "DELETE":
            response = requests.delete(url, headers=HEADERS)
        else:
            raise ValueError(f"Unsupported HTTP method: {method}")
        
        print(f"{method.upper()} {url}")
        print(f"Status Code: {response.status_code}")
        return response
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None

print("Dependencies imported and helper functions defined")

## 2. Basic Connectivity and Health Checks

Before testing the core functionality, we'll verify that the API is running and accessible.

### API Root Endpoint

In [ ]:
print("=== Test 1: API Root Endpoint ===")
response = make_request("GET", "/")

if response and response.status_code == 200:
    root_data = response.json()
    print("API is accessible")
    print("API Information:")
    pprint(root_data)
else:
    print("API root endpoint failed")
    if response:
        print(f"Error: {response.text}")

### Health Check Endpoint

In [ ]:
print("\n=== Test 2: Health Check Endpoint ===")
response = make_request("GET", "/health")

if response and response.status_code == 200:
    health_data = response.json()
    print("Health check passed")
    print("Health Information:")
    pprint(health_data)
    
    # Store initial book count for reference
    initial_book_count = health_data.get('books_loaded', 0)
    print(f"Initial books loaded: {initial_book_count}")
else:
    print("Health check failed")
    if response:
        print(f"Error: {response.text}")

## 3. Book Creation and Validation Tests

Testing the `POST /books/` endpoint with various scenarios:
- Valid book creation
- Data validation (year constraints, required fields)
- Edge cases and error handling

### Valid Book Creation

In [ ]:
print("=== Test 3: Create Valid Books ===")

# Test data for valid books
valid_books = [
    {
        "title": "Test Book 1",
        "author": "Test Author 1",
        "year": 2023,
        "tags": ["fiction", "test"]
    },
    {
        "title": "Test Book 2",
        "author": "Test Author 2", 
        "year": 1950,
        "tags": ["classic"]
    },
    {
        "title": "Test Book Without Tags",
        "author": "Test Author 3",
        "year": 2000
        # No tags field - should default to empty list
    }
]

created_books = []

for i, book_data in enumerate(valid_books, 1):
    print(f"\n--- Creating Book {i} ---")
    response = make_request("POST", "/books/", data=book_data)
    
    if response and response.status_code == 201:
        created_book = response.json()
        created_books.append(created_book)
        print(f"Book created successfully with ID: {created_book['id']}")
        print(f"Title: {created_book['title']}")
        print(f"Author: {created_book['author']}")
        print(f"Year: {created_book['year']}")
        print(f"Tags: {created_book.get('tags', [])}")
    else:
        print(f"Failed to create book {i}")
        if response:
            print(f"Error: {response.text}")

print(f"\nSuccessfully created {len(created_books)} books")

### Validation Tests - Invalid Data

In [ ]:
print("=== Test 4: Data Validation Tests ===")

# Test cases for validation failures
invalid_books = [
    {
        "name": "Missing Required Fields",
        "data": {"title": "Incomplete Book"},  # Missing author and year
        "expected_status": 422
    },
    {
        "name": "Year Too Old",
        "data": {"title": "Ancient Book", "author": "Old Author", "year": 1300},
        "expected_status": 422
    },
    {
        "name": "Year Too New",
        "data": {"title": "Future Book", "author": "Time Traveler", "year": datetime.now().year + 1},
        "expected_status": 422
    },
    {
        "name": "Empty Title",
        "data": {"title": "", "author": "Some Author", "year": 2000},
        "expected_status": 422
    },
    {
        "name": "Empty Author",
        "data": {"title": "Some Title", "author": "", "year": 2000},
        "expected_status": 422
    },
    {
        "name": "Extra Fields",
        "data": {"title": "Valid Title", "author": "Valid Author", "year": 2000, "invalid_field": "should be rejected"},
        "expected_status": 422
    }
]

for test_case in invalid_books:
    print(f"\n--- Testing: {test_case['name']} ---")
    response = make_request("POST", "/books/", data=test_case['data'])
    
    if response is None:
        print(f"Request failed - no response received")
        continue
        
    print(f"Expected: {test_case['expected_status']}, Got: {response.status_code}")
    
    if response.status_code == test_case['expected_status']:
        print(f"Validation correctly rejected invalid data")
        try:
            error_detail = response.json()
            print(f"Error details: {error_detail.get('detail', 'No details provided')}")
        except:
            print(f"Raw response: {response.text}")
    else:
        print(f"Unexpected response for {test_case['name']}")
        print(f"Expected status: {test_case['expected_status']}, Got: {response.status_code}")
        print(f"Response: {response.text}")

### Edge Case Testing

In [ ]:
print("=== Test 5: Edge Cases for Book Creation ===")

edge_cases = [
    {
        "name": "Minimum Valid Year",
        "data": {"title": "Very Old Book", "author": "Ancient Author", "year": 1400}
    },
    {
        "name": "Current Year",
        "data": {"title": "Current Year Book", "author": "Modern Author", "year": datetime.now().year}
    },
    {
        "name": "Very Long Title",
        "data": {"title": "A" * 500, "author": "Long Title Author", "year": 2020}  # Max length
    },
    {
        "name": "Very Long Author Name",
        "data": {"title": "Long Author Book", "author": "B" * 200, "year": 2020}  # Max length
    },
    {
        "name": "Many Tags",
        "data": {"title": "Tagged Book", "author": "Tag Author", "year": 2020, "tags": [f"tag{i}" for i in range(10)]}
    }
]

edge_case_books = []

for test_case in edge_cases:
    print(f"\n--- Testing: {test_case['name']} ---")
    response = make_request("POST", "/books/", data=test_case['data'])
    
    if response and response.status_code == 201:
        created_book = response.json()
        edge_case_books.append(created_book)
        print(f"Edge case handled successfully - Book ID: {created_book['id']}")
    else:
        print(f"Edge case failed: {test_case['name']}")
        if response:
            print(f"Status: {response.status_code}, Error: {response.text}")

print(f"\nSuccessfully created {len(edge_case_books)} edge case books")

## 4. Book Retrieval and Filtering Tests

Testing the `GET /books/` endpoint with various filtering options:
- Basic listing without filters
- Author filtering (case-insensitive)
- Year filtering (exact match)
- Title search (case-insensitive)
- Combined filters

### Basic Book Listing

In [ ]:
print("=== Test 6: Basic Book Listing ===")

response = make_request("GET", "/books/")

if response and response.status_code == 200:
    books_data = response.json()
    print("Successfully retrieved books")
    print(f"Total books: {books_data['total']}")
    print(f"Page: {books_data['page']}")
    print(f"Items per page: {books_data['limit']}")
    print(f"Has next page: {books_data['has_next']}")
    print(f"Has previous page: {books_data['has_prev']}")
    
    print("\n--- First few books ---")
    for book in books_data['items'][:3]:
        print(f"ID: {book['id']}, Title: {book['title']}, Author: {book['author']}, Year: {book['year']}")
        
    # Store total count for later tests
    total_books = books_data['total']
    print(f"\nTotal books in system: {total_books}")
else:
    print("Failed to retrieve books")
    if response:
        print(f"Error: {response.text}")

### Author Filtering Tests

In [ ]:
print("=== Test 7: Author Filtering Tests ===")

# Test case-insensitive author filtering
author_tests = [
    {"author": "Test Author 1", "description": "Exact match"},
    {"author": "test author 1", "description": "Lowercase"},
    {"author": "TEST AUTHOR 1", "description": "Uppercase"},
    {"author": "Test", "description": "Partial match"},
    {"author": "NonExistent Author", "description": "No matches expected"}
]

for test in author_tests:
    print(f"\n--- Testing author filter: {test['description']} ---")
    params = {"author": test["author"]}
    response = make_request("GET", "/books/", params=params)
    
    if response and response.status_code == 200:
        books_data = response.json()
        print(f"Author filter '{test['author']}' returned {books_data['total']} books")
        
        # Show matching books
        for book in books_data['items']:
            print(f"  - {book['title']} by {book['author']}")
    else:
        print(f"Author filtering failed for '{test['author']}'")
        if response:
            print(f"Error: {response.text}")

### Year and Title Search Filtering

In [ ]:
print("=== Test 8: Year and Title Search Filtering ===")

# Year filtering tests
print("\n--- Year Filtering ---")
year_tests = [2023, 1950, 2000, 1999]  # Mix of existing and non-existing years

for year in year_tests:
    params = {"year": year}
    response = make_request("GET", "/books/", params=params)
    
    if response and response.status_code == 200:
        books_data = response.json()
        print(f"Year {year}: Found {books_data['total']} books")
        for book in books_data['items']:
            print(f"  - {book['title']} ({book['year']})")
    else:
        print(f"Year filtering failed for {year}")

# Title search tests
print("\n--- Title Search Filtering ---")
search_tests = [
    {"search": "Test", "description": "Search for 'Test'"},
    {"search": "book", "description": "Search for 'book' (case-insensitive)"},
    {"search": "1984", "description": "Search for '1984'"},
    {"search": "NonExistentTitle", "description": "Search with no matches"}
]

for test in search_tests:
    params = {"search": test["search"]}
    response = make_request("GET", "/books/", params=params)
    
    if response and response.status_code == 200:
        books_data = response.json()
        print(f"{test['description']}: Found {books_data['total']} books")
        for book in books_data['items'][:3]:  # Show first 3 matches
            print(f"  - {book['title']}")
    else:
        print(f"Title search failed for '{test['search']}'")

### Combined Filtering Tests

In [ ]:
print("=== Test 9: Combined Filtering Tests ===")

combined_tests = [
    {
        "params": {"author": "Test", "year": 2023},
        "description": "Author and Year filter"
    },
    {
        "params": {"search": "Test", "author": "Test Author"},
        "description": "Title search and Author filter"
    },
    {
        "params": {"author": "George", "year": 1949},
        "description": "Specific author and year (should find 1984)"
    },
    {
        "params": {"search": "Book", "year": 2000},
        "description": "Title search with specific year"
    }
]

for test in combined_tests:
    print(f"\n--- Testing: {test['description']} ---")
    response = make_request("GET", "/books/", params=test['params'])
    
    if response and response.status_code == 200:
        books_data = response.json()
        print(f"Combined filter returned {books_data['total']} books")
        print(f"Parameters: {test['params']}")
        
        for book in books_data['items']:
            print(f"  - {book['title']} by {book['author']} ({book['year']})")
    else:
        print(f"Combined filtering failed")
        if response:
            print(f"Error: {response.text}")

## 5. Pagination and Sorting Tests

Testing pagination parameters and sorting functionality:
- Different page sizes
- Page navigation
- Sorting by year and author
- Ascending and descending order

### Pagination Tests

In [ ]:
print("=== Test 10: Pagination Tests ===")

# Test different page sizes
page_size_tests = [1, 5, 10, 25, 50, 100]

for limit in page_size_tests:
    print(f"\n--- Testing page size: {limit} ---")
    params = {"limit": limit, "page": 1}
    response = make_request("GET", "/books/", params=params)
    
    if response and response.status_code == 200:
        books_data = response.json()
        actual_items = len(books_data['items'])
        print(f"Requested {limit} items, got {actual_items}")
        print(f"Total: {books_data['total']}, Has next: {books_data['has_next']}")
    else:
        print(f"Pagination failed for limit {limit}")

# Test page navigation
print("\n--- Testing Page Navigation ---")
params = {"limit": 5, "page": 1}
page = 1

while True:
    print(f"\n--- Page {page} ---")
    params["page"] = page
    response = make_request("GET", "/books/", params=params)
    
    if response and response.status_code == 200:
        books_data = response.json()
        print(f"Page {page}: {len(books_data['items'])} items")
        
        # Show book titles on this page
        for book in books_data['items']:
            print(f"  - {book['title']}")
        
        if not books_data['has_next'] or page >= 3:  # Limit to first 3 pages for demo
            print(f"Reached end or page limit (page {page})")
            break
        page += 1
    else:
        print(f"Failed to retrieve page {page}")
        break

### Sorting Tests

In [ ]:
print("=== Test 11: Sorting Tests ===")

sorting_tests = [
    {"sort_by": "year", "sort_order": "asc", "description": "Year ascending"},
    {"sort_by": "year", "sort_order": "desc", "description": "Year descending"},
    {"sort_by": "author", "sort_order": "asc", "description": "Author ascending"},
    {"sort_by": "author", "sort_order": "desc", "description": "Author descending"}
]

for test in sorting_tests:
    print(f"\n--- Testing: {test['description']} ---")
    params = {
        "sort_by": test["sort_by"],
        "sort_order": test["sort_order"],
        "limit": 10
    }
    
    response = make_request("GET", "/books/", params=params)
    
    if response and response.status_code == 200:
        books_data = response.json()
        print(f"Sorting by {test['sort_by']} ({test['sort_order']}) successful")
        
        # Display sorted results
        for book in books_data['items']:
            if test["sort_by"] == "year":
                print(f"  - {book['title']} ({book['year']})")
            else:  # author
                print(f"  - {book['title']} by {book['author']}")
                
        # Verify sorting is correct
        if books_data['items']:
            sort_values = [book[test["sort_by"]] for book in books_data['items']]
            if test["sort_by"] == "author":
                sort_values = [val.lower() for val in sort_values]
            
            is_sorted = all(sort_values[i] <= sort_values[i+1] for i in range(len(sort_values)-1))
            if test["sort_order"] == "desc":
                is_sorted = all(sort_values[i] >= sort_values[i+1] for i in range(len(sort_values)-1))
            
            if is_sorted:
                print(f"  Results are correctly sorted")
            else:
                print(f"  Results are NOT correctly sorted")
                print(f"  Values: {sort_values}")
    else:
        print(f"Sorting failed for {test['description']}")
        if response:
            print(f"Error: {response.text}")

### Invalid Pagination and Sorting Parameters

In [ ]:
print("=== Test 4: Data Validation Tests ===")

# Test cases for validation failures
invalid_books = [
    {
        "name": "Missing Required Fields",
        "data": {"title": "Incomplete Book"},  # Missing author and year
    },
    {
        "name": "Year Too Old",
        "data": {"title": "Ancient Book", "author": "Old Author", "year": 1300},
    },
    {
        "name": "Year Too New",
        "data": {"title": "Future Book", "author": "Time Traveler", "year": datetime.now().year + 1},
    },
    {
        "name": "Empty Title",
        "data": {"title": "", "author": "Some Author", "year": 2000},
    },
    {
        "name": "Empty Author",
        "data": {"title": "Some Title", "author": "", "year": 2000},
    },
    {
        "name": "Extra Fields",
        "data": {"title": "Valid Title", "author": "Valid Author", "year": 2000, "invalid_field": "should be rejected"},
    }
]

for test_case in invalid_books:
    print(f"\n--- Testing: {test_case['name']} ---")
    
    # Make request directly
    url = f"{BASE_URL}/books/"
    try:
        response = requests.post(url, json=test_case['data'], headers=HEADERS)
        print(f"POST {url}")
        print(f"Status Code: {response.status_code}")
        
        # Check if validation worked correctly
        if response.status_code == 422:
            print(f"Validation correctly rejected invalid data")
            try:
                error_detail = response.json()
                print(f"Validation errors: {error_detail.get('detail', 'No details provided')}")
            except:
                print(f"Raw response: {response.text}")
                
        elif response.status_code == 400:
            print(f"Invalid data rejected as bad request")
            try:
                error_detail = response.json()
                print(f"Error: {error_detail.get('detail', 'No details provided')}")
            except:
                print(f"Raw response: {response.text}")
                
        else:
            print(f"Unexpected response for {test_case['name']}")
            print(f"Expected 422, Got: {response.status_code}")
            print(f"Response: {response.text}")
            
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")

print(f"\n📊 Data validation tests completed")

## 6. Individual Book Operations

Testing operations on specific books:
- Retrieving books by ID
- Deleting books by ID
- Error handling for non-existent IDs

### Get Book by ID Tests

In [ ]:
print("=== Test 13: Get Book by ID Tests ===")

# First, get a list of existing book IDs
url = f"{BASE_URL}/books/"
try:
    response = requests.get(url, params={"limit": 5}, headers=HEADERS)
    existing_ids = []
    
    if response and response.status_code == 200:
        books_data = response.json()
        existing_ids = [book['id'] for book in books_data['items']]
        print(f"Found existing book IDs: {existing_ids}")
    else:
        print("Could not retrieve existing books for testing")
        existing_ids = []

except requests.exceptions.RequestException as e:
    print(f"Failed to get existing books: {e}")
    existing_ids = []

# Test retrieving existing books
if existing_ids:
    for book_id in existing_ids[:3]:  # Test first 3 books
        print(f"\n--- Getting book ID: {book_id} ---")
        
        try:
            url = f"{BASE_URL}/books/{book_id}"
            response = requests.get(url, headers=HEADERS)
            print(f"GET {url}")
            print(f"Status Code: {response.status_code}")
            
            if response.status_code == 200:
                book = response.json()
                print(f"Successfully retrieved book {book_id}")
                print(f"Title: {book['title']}")
                print(f"Author: {book['author']}")
                print(f"Year: {book['year']}")
                print(f"Tags: {book.get('tags', [])}")
            else:
                print(f"Failed to retrieve book {book_id}")
                print(f"Response: {response.text}")
                
        except requests.exceptions.RequestException as e:
            print(f"Request failed for book {book_id}: {e}")

# Test non-existent book IDs
print("\n--- Testing Non-existent Book IDs ---")
non_existent_ids = [99999, -1, 0]

for book_id in non_existent_ids:
    print(f"\n--- Getting non-existent book ID: {book_id} ---")
    
    try:
        url = f"{BASE_URL}/books/{book_id}"
        response = requests.get(url, headers=HEADERS)
        print(f"GET {url}")
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 404:
            print(f"Correctly returned 404 for non-existent book {book_id}")
            
            # Try to get error message with better handling
            try:
                error = response.json()
                print(f"Response JSON: {error}")  # Debug: see full response
                
                # Try different possible field names
                error_msg = (error.get('detail') or 
                           error.get('message') or 
                           error.get('error') or 
                           'No error message provided')
                print(f"Error message: {error_msg}")
                
            except Exception as e:
                print(f"Could not parse JSON response: {e}")
                print(f"Raw response: '{response.text}'")
                
        else:
            print(f"Unexpected response for non-existent book {book_id}")
            print(f"Expected 404, Got: {response.status_code}")
            print(f"Response: {response.text}")
            
    except requests.exceptions.RequestException as e:
        print(f"Request failed for book {book_id}: {e}")

print(f"\nBook retrieval tests completed")

### Delete Book Tests

In [ ]:
print("=== Test 14: Delete Book Tests ===")

# Create test books for deletion
books_to_delete = [
    {"title": "Delete Test 1", "author": "Delete Author 1", "year": 2023},
    {"title": "Delete Test 2", "author": "Delete Author 2", "year": 2023}
]

deletion_test_ids = []

print("--- Creating books for deletion testing ---")
for book_data in books_to_delete:
    response = requests.post(f"{BASE_URL}/books/", json=book_data, headers=HEADERS)
    if response.status_code == 201:
        book = response.json()
        deletion_test_ids.append(book['id'])
        print(f"Created book for deletion: ID {book['id']}")

# Test successful deletion
print("\n--- Testing Book Deletion ---")
for book_id in deletion_test_ids:
    print(f"\n--- Deleting book ID: {book_id} ---")
    
    # Delete the book
    delete_response = requests.delete(f"{BASE_URL}/books/{book_id}", headers=HEADERS)
    print(f"DELETE /books/{book_id} -> Status: {delete_response.status_code}")
    
    if delete_response.status_code == 200:
        print(f"Successfully deleted book {book_id}")
        
        # Verify it's gone
        verify_response = requests.get(f"{BASE_URL}/books/{book_id}", headers=HEADERS)
        if verify_response.status_code == 404:
            print(f"Verified: Book {book_id} is deleted")
        else:
            print(f"ERROR: Book {book_id} still exists after deletion!")
    else:
        print(f"Failed to delete book {book_id}")

# Test deleting non-existent books
print("\n--- Testing Non-existent Book Deletion ---")
for book_id in [99999, -1]:
    response = requests.delete(f"{BASE_URL}/books/{book_id}", headers=HEADERS)
    print(f"DELETE /books/{book_id} -> Status: {response.status_code}")
    
    if response.status_code == 404:
        print(f"Correctly returned 404 for non-existent book {book_id}")
    else:
        print(f"Unexpected response for non-existent book {book_id}")

print("\nDelete book tests completed")

## 7. Statistics Endpoint Tests

Testing the `/books/stats/summary` endpoint:
- Basic statistics retrieval
- Verification of counts
- Statistics accuracy after CRUD operations

### Statistics Tests

In [ ]:
print("=== Test 15: Statistics Endpoint Tests ===")

# Get current statistics
print("--- Current Statistics ---")
response = make_request("GET", "/books/stats/summary")

if response and response.status_code == 200:
    stats = response.json()
    print("Successfully retrieved statistics")
    print(f"Total books: {stats['total_books']}")
    print(f"Unique authors: {stats['unique_authors']}")
    
    # Store current stats for verification
    current_total = stats['total_books']
    current_authors = stats['unique_authors']
else:
    print("Failed to retrieve statistics")
    if response:
        print(f"Error: {response.text}")
    current_total = 0
    current_authors = 0

# Verify statistics by manually counting
print("\n--- Verifying Statistics Accuracy ---")
response = make_request("GET", "/books/", params={"limit": 100})  # Get many books

if response and response.status_code == 200:
    books_data = response.json()
    manual_total = books_data['total']
    
    # Count unique authors manually
    all_authors = set()
    page = 1
    while True:
        page_response = make_request("GET", "/books/", params={"limit": 100, "page": page})
        if page_response and page_response.status_code == 200:
            page_data = page_response.json()
            for book in page_data['items']:
                all_authors.add(book['author'])
            
            if not page_data['has_next']:
                break
            page += 1
        else:
            break
    
    manual_authors = len(all_authors)
    
    print(f"Manual count - Total books: {manual_total}")
    print(f"Manual count - Unique authors: {manual_authors}")
    
    # Compare with stats endpoint
    if current_total == manual_total:
        print("Total books count matches")
    else:
        print(f"Total books mismatch: Stats={current_total}, Manual={manual_total}")
        
    if current_authors == manual_authors:
        print("Unique authors count matches")
    else:
        print(f"Unique authors mismatch: Stats={current_authors}, Manual={manual_authors}")

# Test statistics after creating a new book
print("\n--- Testing Statistics After Book Creation ---")
new_book = {
    "title": "Stats Test Book",
    "author": "Unique Stats Author",
    "year": 2023
}

create_response = make_request("POST", "/books/", data=new_book)
if create_response and create_response.status_code == 201:
    created_book = create_response.json()
    print(f"Created test book with ID: {created_book['id']}")
    
    # Get updated statistics
    stats_response = make_request("GET", "/books/stats/summary")
    if stats_response and stats_response.status_code == 200:
        new_stats = stats_response.json()
        print(f"New total books: {new_stats['total_books']} (was {current_total})")
        print(f"New unique authors: {new_stats['unique_authors']} (was {current_authors})")
        
        # Verify the counts increased
        if new_stats['total_books'] == current_total + 1:
            print("Total books count increased correctly")
        else:
            print("Total books count did not increase correctly")
            
        # Note: Unique authors might or might not increase depending on if the author already exists
        if new_stats['unique_authors'] >= current_authors:
            print("Unique authors count is correct (same or increased)")
        else:
            print("Unique authors count decreased unexpectedly")
        
        # Clean up - delete the test book
        delete_response = make_request("DELETE", f"/books/{created_book['id']}")
        if delete_response and delete_response.status_code == 200:
            print(f"Cleaned up test book {created_book['id']}")

## 8.  Error Handling and Edge Cases

Testing various error scenarios and edge cases:
- Invalid HTTP methods
- Malformed JSON
- Server error simulation
- Boundary conditions

### HTTP Method Tests

In [ ]:
print("=== Test 16: Invalid HTTP Methods ===")

# Test unsupported methods on various endpoints
method_tests = [
    {"method": "PUT", "endpoint": "/books/", "description": "PUT on books collection"},
    {"method": "PATCH", "endpoint": "/books/1", "description": "PATCH on specific book"},
    {"method": "POST", "endpoint": "/books/1", "description": "POST on specific book"},
    {"method": "DELETE", "endpoint": "/books/", "description": "DELETE on books collection"},
]

for test in method_tests:
    print(f"\n--- Testing: {test['description']} ---")
    url = f"{BASE_URL}{test['endpoint']}"
    
    try:
        if test['method'] == 'PUT':
            response = requests.put(url, json={}, headers=HEADERS)
        elif test['method'] == 'PATCH':
            response = requests.patch(url, json={}, headers=HEADERS)
        elif test['method'] == 'POST':
            response = requests.post(url, json={}, headers=HEADERS)
        elif test['method'] == 'DELETE':
            response = requests.delete(url, headers=HEADERS)
        
        print(f"{test['method']} {url} -> Status: {response.status_code}")
        
        if response.status_code == 405:
            print("Correctly returned 405 Method Not Allowed")
        elif response.status_code == 422:
            print("Returned 422 (validation error - also acceptable)")
        else:
            print(f"Unexpected status code: {response.status_code}")
            
    except Exception as e:
        print(f"Request failed: {e}")

### Malformed Request Tests

In [ ]:
print("=== Test 17: Malformed Request Tests ===")

# Test malformed JSON
print("--- Testing Malformed JSON ---")
malformed_tests = [
    '{"title": "Test", "author": "Test"',  # Missing closing brace
    '{"title": "Test", "author": "Test", "year": "not_a_number"}',  # Wrong data type
    '',  # Empty body
    'not json at all'  # Not JSON
]

for i, bad_data in enumerate(malformed_tests, 1):
    print(f"\n--- Test {i}: Malformed JSON ---")
    response = requests.post(
        f"{BASE_URL}/books/", 
        data=bad_data,
        headers={"Content-Type": "application/json"}
    )
    
    print(f"Status: {response.status_code}")
    if response.status_code in [400, 422]:
        print(f"Correctly rejected malformed request")
    else:
        print(f"Unexpected status: {response.status_code}")

# Test missing Content-Type
print(f"\n--- Testing Missing Content-Type ---")
response = requests.post(
    f"{BASE_URL}/books/",
    data='{"title": "Test", "author": "Test", "year": 2023}',
    headers={}  # No Content-Type header
)

print(f"Status: {response.status_code}")
if response.status_code in [400, 415, 422]:
    print(f"Correctly rejected missing Content-Type")
elif response.status_code == 201:
    print(f"API accepted request (FastAPI auto-handling)")
else:
    print(f"Unexpected status: {response.status_code}")

print("\nMalformed request tests completed")